In [ ]:
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import os
import numpy as np
import json

from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.image as img

import cv2
import math

In [ ]:
#기본적으로 label에 해당하는 json파일에 정보가 image양 보다 많다.
#가령 (label File)20201028_cat-arch-000156.mp4.json 파일 안에는 frame_0, frame_3, frame_6,...,frame 258까지 있는데 반해, 
#(Image file)20201028_cat-arch-000156.mp4안의 image는 25개 정도 밖에 없다...
#심지어 json파일내 frame 숫자가 일정한 숫자 간격으로 늘어나는 것도 아니다. 예) frame_0, frame_3 다음 frame숫자는? frame_6이라 단정못함. frame_9일 수 도 있음...
#자세한 정보는 파일 한번 열어보시면, 알 수 있습니다. 로컬 컴퓨터에 데이터 작은거 몇개 열어보시면 보기 더 편하실 듯 합니다.

In [ ]:
#Label파일과 image파일에 약간의? 문제가 있는 관계로 고민을 좀 많이했다.
#그 결과, dataset class getitem에서 image랑 그 image에 맞는 필요한 부분만 뽑아서 사용하려 한다. 

class cnn_dataset(Dataset):
  def __init__(self, label_dir, image_dir, transform=None):
    self.label_dir = label_dir
    self.image_dir = image_dir
    self.transform = transform

    list_image = os.listdir(self.image_dir)
    self.list_image = list_image

  #이미지 하나 당 target 생성
  def __getitem__(self, index):
    image = os.path.join(self.image_dir, self.list_image[index])
    image = Image.open(image).convert('RGB')
    #image = np.asarray(image)

    target = Make_target(self.label_dir, Find_frame_num(self.list_image[index])) #밑에 Make_target함수와 Find_frame_num함수 설명이 있습니다.
    
    if self.transform:
      image = self.transform(image)

    target = torch.tensor(target)

    data = {'image':image, 'target':target}
    return data

  def __len__(self):
    return len(self.list_image)
       

In [ ]:
#yolo용 target 구조: (x, y, w, h, c, cat, dog, i, j)
#만약 CNN 학습을 위해 target에 breed를 추가하신다면, (breed, x, y, w, h, c, cat, dog, i, j)

#Image에 필요한 target을 만들려고함
#근데 존나 마음아픈게 frame별로 걍 마구잡이로 되 있어서, Image파일 이름에 있는 frame 숫자를 이용해서 멥핑. /ARCH/20201028_cat-arch-000156.mp4/frame_102_timestamp_0.jpg => 102  
#이 함수는 라벨(json)과 frame_num을 인자로 받아 해당 frame에 대한 bounding box, speices, responsible grid cell을 찾는다. *추가: 만약 breed가 추가된다면 breed도...
def Make_target(label_dir, frame_num):
  with open(label_dir) as f:
    label_json = json.load(f)
  
  s = 8

  target_list = []
  #바운딩박스 찾기
  target_list.append(label_json['metadata']['breed'])
  for i in range(len(label_json['annotations'])):
    if label_json['annotations'][i]['frame_number'] == frame_num:
       target_list.append(label_json['annotations'][i]['bounding_box']['x'])
       target_list.append(label_json['annotations'][i]['bounding_box']['y'])
       target_list.append(label_json['annotations'][i]['bounding_box']['width'])
       target_list.append(label_json['annotations'][i]['bounding_box']['height'])
  
  #바운딩박스 yolo의 target 형식에 맞게 변경
  image_width = label_json['metadata']['width']
  image_height = label_json['metadata']['height']

  s_image_width = image_width / 8
  s_image_height = image_height / 8

  i, j = 0, 0
  while(i*s_image_width < target_list[0]):
    i += 1

  while(j*s_image_height < target_list[1]):
    j += 1

  i -= 1
  j -= 1
  
  target_list[0] = target_list[0]/s_image_width - i
  target_list[1] = target_list[1]/s_image_height - j
  target_list[2] /= s_image_width
  target_list[3] /= s_image_height


  #confidence score를 1로 설정
  target_list.append(1.0)

  #고양이면 1, 0 을 강아지면 0,1을 append
  if label_json['metadata']['species'] == 'CAT':
    target_list.append(1.0)
    target_list.append(0)
  else:
    target_list.append(0)
    target_list.append(1.0)

  target_list.append(i)
  target_list.append(j)
  
  return target_list  

In [ ]:
#이미지 파일 이름을 이용해서, frame 숫자를 뽑아내는 함수이다.
def Find_frame_num(image_filename):
  frame_num = ''
  count_ = 0
  

  for i in range(len(image_filename)):
    if image_filename[i] == '_':
      count_ += 1
      if count_ == 2:
        break
      continue
    if count_ == 1:
      frame_num += image_filename[i]
  
  frame_num = int(frame_num)
  return frame_num
